In [9]:
from keras.models import load_model
from keras import models
model = load_model("Fine_Tuning.h5")

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_15 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 257       
Total params: 22,122,049
Trainable params: 6,817,281
Non-trainable params: 15,304,768
_________________________________________________________________


In [11]:
layer = model.get_layer("dense_29")

In [12]:
layer_output = layer.output

In [13]:
activation_model = models.Model(input=model.input, outputs=[layer_output])

C:\Users\gabal\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("vg...)`
  """Entry point for launching an IPython kernel.


In [ ]:
activations = activation_model.predict(x_test)